This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'QkAcqomCtz4mzHYsXGA3'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key={}'.format(API_KEY))

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json();

From this output, we can see that the core metadata of AFX_X is at the second level and the daily stock info is stored within this level as a list with the key 'data'. There is also a list of column names that will be useful.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Question 1 & 2

Questions 1 and 2 will be done through a single dictionary comprehension to extract all 2017 data into a new dictionary.

In [5]:
# create dict with date as key and daily stock info as values
afx_dict = {date[0] : date[1:] for date in r.json()['dataset']['data'] if '2017' in date[0]}

# extract list of column names from json object skipping the date
val_names = r.json()['dataset']['column_names'][1:]

# print out list of value names
print(f'Here is the list of value names: {val_names}')

Here is the list of value names: ['Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


## Question 3

From looking at the daily stock info in afx_dict, there are some instances that have None rather than values. For now these values will be skipped rather than imputed since we're looking for the max and min opening values.

In [6]:
# function to find stock value based on aggregate method
def find_value(name, method):
    # list comprehension to store stock values
    daily = [afx_dict[day][val_names.index(name)] 
             for day in afx_dict 
             if afx_dict[day][val_names.index(name)] is not None]
    return method(daily)

In [7]:
# extract maximum and minimum opening values
max_open = find_value('Open', max)
min_open = find_value('Open', min)

# print out the highest and lowest opening value for 2017
print(f'The highest opening value in 2017 was {max_open} and the lowest opening value was {min_open}')

The highest opening value in 2017 was 53.11 and the lowest opening value was 34.0


## Question 4

In [8]:
# calculate daily change by subtracting daily low from daily high
daily_change = [afx_dict[day][val_names.index('High')] - afx_dict[day][val_names.index('Low')] 
                for day in afx_dict 
                if afx_dict[day][val_names.index('High')] and afx_dict[day][val_names.index('Low')] is not None]

# print out maximum daily change value
print(f'The largest daily change of stock price during 2017 was {max(daily_change)}')

The largest daily change of stock price during 2017 was 2.8100000000000023


## Question 5

In [9]:
# list of keys representing days in 2017
day_list = list(afx_dict.keys())

# list of between day change values
change = []

# counter to hold the key index for each day
day_counter = 0

# loop through days
### NOTE that len(day_list) - 1 because python is zero indexed
while day_counter < len(day_list) - 1:
    # calculate change
    change.append(afx_dict[day_list[day_counter]][val_names.index('Close')] - 
                  afx_dict[day_list[day_counter + 1]][val_names.index('Close')])   
    # increase counter by one
    day_counter += 1

# print out max between day change
print(f'The maximum change in stock price between two days for 2017 is {max(change)}')

The maximum change in stock price between two days for 2017 is 1.7199999999999989


## Question 6

In [10]:
# import core statistics module
import statistics

# print average daily trading volume
print(f'The average daily trading volume in 2017 was {find_value("Traded Volume", statistics.mean)}')

The average daily trading volume in 2017 was 89124.33725490196


## Question 7

In [11]:
# print average daily trading volume
print(f'The average daily trading volume in 2017 was {find_value("Traded Volume", statistics.median)}')

The average daily trading volume in 2017 was 76286.0
